In [1]:
import codes2network
import json
import random

data = "../data/TRIP_bincoded_201704.csv"

node_names, node_weights, edge_dict = codes2network.csv2network(data)

Processed 10165 articles


In [2]:
edge_dict

{(39, 70): ['2004', 0.3049680275454993],
 (80, 125): ['1988', 0.009837678307919331],
 (100, 112): ['1980', 0.5312346286276439],
 (63, 76): ['1995', 0.8263649778652238],
 (33, 41): ['2000', 0.6197737333989178],
 (15, 30): ['2007', 0.05902606984751599],
 (56, 101): ['2006', 0.029513034923757994],
 (19, 91): ['2001', 1.9773733398917854],
 (76, 88): ['2007', 0.4722085587801279],
 (42, 88): ['2006', 0.1278898180029513],
 (5, 84): ['2004', 0.05902606984751599],
 (8, 87): ['2006', 0.18691588785046728],
 (85, 87): ['1991', 0.49188391539596654],
 (32, 77): ['2006', 0.23610427939006395],
 (14, 77): ['1990', 1.5641908509591735],
 (71, 100): ['1987', 3.0791933103787508],
 (18, 112): ['1988', 0.07870142646335465],
 (75, 97): ['2005', 0.05902606984751599],
 (38, 119): ['2010', 1.9380226266601082],
 (4, 35): ['2001', 0.2852926709296606],
 (8, 38): ['2000', 1.6724053123462863],
 (61, 63): ['2000', 0.6492867683226758],
 (27, 115): ['1981', 0.009837678307919331],
 (47, 124): ['2005', 0.11805213969503198

We take the edge dictionary and make a dictionary that has the years as keys. 

In [4]:
yr_dict = {}
# structure of x: ((connection tuple), [year, strength of connection])
for x in edge_dict.iteritems():
    yr = x[1][0]
    if yr in yr_dict:
        yr_dict[yr].append([x[0][0], x[0][1], x[1][1]])
    else:
        yr_dict[yr] = [[x[0][0], x[0][1], x[1][1]]]
        
yr_dict

{'1980': [[100, 112, 0.5312346286276439],
  [17, 57, 0.05902606984751599],
  [26, 74, 0.2852926709296606],
  [31, 108, 0.4131824889326119],
  [33, 53, 0.3049680275454993],
  [3, 103, 0.8755533694048204],
  [7, 104, 0.5410723069355632],
  [43, 75, 1.1706837186424004],
  [108, 126, 0.07870142646335465],
  [50, 75, 0.5213969503197246],
  [7, 76, 0.4426955238563699],
  [74, 121, 0.4131824889326119],
  [35, 76, 0.4131824889326119],
  [10, 103, 1.013280865715691],
  [71, 103, 2.0068863748155437],
  [3, 10, 0.37383177570093457],
  [64, 108, 1.0329562223315298],
  [26, 75, 0.06886374815543532],
  [35, 104, 0.3049680275454993],
  [76, 125, 0.46237088047220853],
  [50, 55, 3.3841613379242497],
  [46, 59, 0.009837678307919331],
  [11, 108, 0.14756517461878996],
  [53, 108, 5.7255287752090505],
  [47, 70, 0.039350713231677326],
  [3, 17, 0.7181505164781111],
  [50, 108, 3.649778652238072],
  [1, 77, 0.8263649778652238],
  [33, 47, 0.019675356615838663],
  [76, 90, 2.3807181505164783],
  [3, 13, 0.

In [5]:
def network2nodes(outfile_path, yr_dict, keep_strengths = False):
    
    """
    Params:
    
    String data: File path to the Trip database
    
    String outfile_path: File path to the file that you want to write to. Will be read by D3 force graphs
    
    Dictionary yr_dict: A dictionary with yrs as keys, edge information as keys. This information includes 
    what node(s) it attaches too, and the strength of that attachment
    
    Bool keep_strengths: boolean flag on whether to keep the strength of a connection as part of the data
    
    Initial structure of just_connects
    {
        "2000": {0: [......], 1: [.....]},
        "2001": {0: [......], 1: [.....]}
    }
    
    We then strip the index keys (0, 1, 2, ... etc.) from the inner dictionaries so that the final structure
    is as follows:
    {
        "2000": [[.....],[.....]],
        "2001": [[.....],[.....]]
    }
    
    For each year as a key, there is a list of nodes, denoted by the index of the list. At each index there is
    a sublist with indices of the nodes that that node is connected to. For example, if we wanted to see what
    node 50 was connected to in 2015, we would type connects['2015'][50]. This returns a sublist of other 
    indices that node 50 is connected too.
    """
    connects = {}

    for key in yr_dict.iterkeys():
        connects[key] = {}
        for connection in yr_dict[key]:
            if connection[0] in connects[key]:
                if keep_strengths:
                    #append tuple including strength
                    connects[key][connection[0]].append((connection[1], connection[2]))
                else:
                    connects[key][connection[0]].append(connection[1])
            else:
                if keep_strengths:
                    connects[key][connection[0]] = [(connection[1], connection[2])]
                else:
                    connects[key][connection[0]] = [connection[1]]

    
    #strip the numeric index key out so that we are left with only years and their values
    for key in connects.iterkeys():
        connects[key] = connects[key].values()
    
    #Check the indexes in each list and make sure they don't point to an index that is out of range
    for key,val in connects.iteritems():
        for node in xrange(len(val)):
            if keep_strengths:
                filtered_list = [connection for connection in val[node] if connection[0] < len(val)]
            else:
                filtered_list = [connection for connection in val[node] if connection < len(val)]
            val[node] = filtered_list
            
    #save file as a json
    with open(outfile_path, 'w') as outfile:
        json.dump(connects, outfile)
        
    return connects


In [6]:
with_strengths = network2nodes('../data/years_with_strength.json', yr_dict, keep_strengths=True)

In [7]:
with_strengths

{'1980': [[(19, 0.5213969503197246),
   (41, 3.541564190850959),
   (46, 0.4131824889326119),
   (20, 1.1510083620265616),
   (14, 1.042793900639449),
   (3, 1.0821446138711264),
   (38, 2.695523856369897)],
  [(57, 0.6296114117068372), (53, 16.22233152975898)],
  [(10, 0.37383177570093457),
   (17, 0.7181505164781111),
   (13, 0.09837678307919331),
   (54, 0.2754549926217413),
   (57, 0.19675356615838663),
   (26, 0.2951303492375799)],
  [(44, 0.6787998032464339), (53, 1.1805213969503197)],
  [(55, 0.6000983767830792)],
  [(53, 5.9222823413674375)],
  [(57, 0.2557796360059026)],
  [],
  [(52, 0.13772749631087064)],
  [(55, 1.515002459419577),
   (53, 3.4530250860796854),
   (57, 0.09837678307919331)],
  [(57, 0.05902606984751599), (54, 0.2852926709296606)],
  [(53, 2.05607476635514)],
  [],
  [(44, 0.39350713231677326), (53, 2.9021151008362027)],
  [(54, 0.2951303492375799)],
  [(36, 0.009837678307919331)],
  [(36, 0.009837678307919331)],
  [(57, 0.019675356615838663), (55, 0.29513034

In [9]:
no_strength = network2nodes('../data/years.json', yr_dict, keep_strengths=False)

In [10]:
no_strength

{'1980': [[19, 41, 46, 20, 14, 3, 38],
  [57, 53],
  [10, 17, 13, 54, 57, 26],
  [44, 53],
  [55],
  [53],
  [57],
  [],
  [52],
  [55, 53, 57],
  [57, 54],
  [53],
  [],
  [44, 53],
  [54],
  [36],
  [36],
  [57, 55],
  [55, 47],
  [57, 53],
  [53, 55],
  [57],
  [53, 47],
  [],
  [57, 54, 53],
  [53, 55],
  [53, 57],
  [55, 53, 57],
  [53, 57],
  [57, 53],
  [52],
  [59, 55, 53],
  [],
  [54, 53],
  [55],
  [59],
  [],
  [58],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 '1981': [[3],
  [46, 14, 20, 28, 30, 5, 40, 43],
  [],
  [],
  [45],
  [],
  [45, 44],
  [15],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 '1982': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],

make dictionary with topics as key, values are list of tuples that node connects to with its weight.

In [68]:
just_connects

{'2000': [[75, 46, 53, 55, 20, 88],
  [27, 128, 134, 127, 119],
  [123, 120, 33, 58, 76, 21, 39],
  [13, 69, 18, 61, 63, 118, 33, 88, 4, 43, 99, 95, 94],
  [14, 43, 120, 92, 16, 103, 19, 70, 24, 131, 67, 33, 78],
  [105, 74],
  [132, 102, 14, 119, 80, 107],
  [78, 30, 120, 122, 18, 118, 90, 112, 87, 36],
  [39, 64, 102],
  [124, 17, 49, 45, 82, 28, 118],
  [25, 77, 118, 34, 80, 88, 68, 120],
  [56, 108, 111, 38, 61, 69, 12, 112, 78],
  [91, 122, 27, 79, 109, 110, 55, 134, 15, 64, 40, 67],
  [48, 83, 33, 91],
  [98, 19, 110, 111],
  [122, 40, 89, 68, 42, 22, 134, 60],
  [69, 130],
  [116, 29, 104, 72, 131, 132, 115],
  [111, 67, 34, 27, 52, 69, 132, 87],
  [64, 92, 78, 111, 84, 112, 31],
  [26, 63, 128],
  [37, 36],
  [73, 113, 29, 78, 64, 99],
  [47, 81, 38, 107, 61],
  [106, 36],
  [36, 46, 44, 96, 124, 56, 26, 32],
  [94, 62, 51, 77, 101, 90],
  [86, 104, 105, 77, 34, 78, 98, 48, 51, 88],
  [89, 81],
  [98, 97, 101, 65, 52, 74, 111, 56, 117],
  [36, 84, 75, 89, 68, 98, 50],
  [108, 1

**Problem:** The index of the node is supposed to be the id of the node, and the sublists at that index are supposed to point to other node ids. However, some of the node ids point to indexes that are outside of the range of the list. For example, at index 2, we could have a list like [27, 128, 134, 127, 119], but the full list is only of size 116. So when the network is being constructed, it will look at index 2 (node 2) and think that it's connected to node 128, but that node is not in our network.

In [69]:
len(just_connects['2000'])

116

In [74]:
just_connects

{'2000': [[75, 46, 53, 55, 20, 88],
  [27],
  [33, 58, 76, 21, 39],
  [13, 69, 18, 61, 63, 33, 88, 4, 43, 99, 95, 94],
  [14, 43, 92, 16, 103, 19, 70, 24, 67, 33, 78],
  [105, 74],
  [102, 14, 80, 107],
  [78, 30, 18, 90, 112, 87, 36],
  [39, 64, 102],
  [17, 49, 45, 82, 28],
  [25, 77, 34, 80, 88, 68],
  [56, 108, 111, 38, 61, 69, 12, 112, 78],
  [91, 27, 79, 109, 110, 55, 15, 64, 40, 67],
  [48, 83, 33, 91],
  [98, 19, 110, 111],
  [40, 89, 68, 42, 22, 60],
  [69],
  [29, 104, 72, 115],
  [111, 67, 34, 27, 52, 69, 87],
  [64, 92, 78, 111, 84, 112, 31],
  [26, 63],
  [37, 36],
  [73, 113, 29, 78, 64, 99],
  [47, 81, 38, 107, 61],
  [106, 36],
  [36, 46, 44, 96, 56, 26, 32],
  [94, 62, 51, 77, 101, 90],
  [86, 104, 105, 77, 34, 78, 98, 48, 51, 88],
  [89, 81],
  [98, 97, 101, 65, 52, 74, 111, 56],
  [36, 84, 75, 89, 68, 98, 50],
  [108, 109, 66, 80, 53, 75, 104],
  [68, 101, 34, 83],
  [91, 46, 87, 35],
  [105, 37, 53],
  [70, 46, 108],
  [102, 98, 58],
  [88, 55],
  [92, 61, 82, 111, 

In [75]:
with open('/Users/benjamindykstra/Documents/dynamic_graphs/data/years.json', 'w') as outfile:
    json.dump(just_connects, outfile)
